# Manual Testing Notebook

This notebook allows you to step through the logic of the Credit Risk Reporting application manually.

In [1]:
import os
import sys
import yaml
import pandas as pd

# Ensure we can import from src
sys.path.append(os.path.abspath('..'))

from src.data_generator import SyntheticDataLoader
from src.calculations import CreditRiskFormulas
from src.ai_engine import RiskReporter

## 1. Load Configuration

In [2]:
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

print("Alpha:", config['thresholds']['significance_level_alpha'])
print("Model:", config['ai_settings']['model'])

Alpha: 0.05
Model: gpt-4o


## 2. Generate Synthetic Data

In [3]:
loader = SyntheticDataLoader()

df_auc = loader.generate_auc_data()
print("AUC Data Head:")
display(df_auc.head())

AUC Data Head:


,Date,AUC_Initial,AUC_Current
0,2025-01-06,0.75,0.759934
1,2025-02-05,0.75,0.747235
2,2025-03-07,0.75,0.762954
3,2025-04-06,0.75,0.780461
4,2025-05-06,0.75,0.745317


In [4]:
df_calib = loader.generate_calibration_data()
print("Calibration Data:")
display(df_calib)

Calibration Data:


,Grade,PD,N,Defaults,Observed_DR
0,A,0.005,1000,4,0.004
1,B,0.010,1500,15,0.010
2,C,0.020,2000,32,0.016
3,D,0.050,1500,69,0.046
4,E,0.100,1000,103,0.103
5,F,0.200,500,101,0.202
6,G,0.300,200,59,0.295


In [5]:
df_scores = loader.generate_score_data()
print("Score Data Head:")
display(df_scores.head())

Score Data Head:


,Score_Dev,Score_Curr
0,572.780864,684.832245
1,605.546129,617.688831
2,542.450321,596.976181
3,618.784901,601.241629
4,569.968066,626.902935


## 3. Calculate Statistics

In [6]:
calculator = CreditRiskFormulas(config)

auc_stats = calculator.evaluate_auc(df_auc)
print("AUC Stats:", auc_stats)

chi_stats = calculator.calculate_chi_square(df_calib)
print("Chi-Square Stats:", chi_stats)

ttest_stats = calculator.calculate_ttest(df_scores)
print("T-Test Stats:", ttest_stats)

AUC Stats: {'Latest_AUC_Diff': np.float64(-0.009314595071405085), 'Status': 'Accept', 'Details': 'Diff -0.0093 vs Tolerance 0.02'}
Chi-Square Stats: {'Chi2_Stat': np.float64(0.3842694876926676), 'P_Value': np.float64(0.535327089530766), 'Status': 'Accept'}
T-Test Stats: {'T_Stat': np.float64(1.7776962560956935), 'P_Value': np.float64(0.07560635446405671), 'Status': 'Accept'}


In [7]:
binomial_results = calculator.calculate_binomial_test(df_calib)
print("Binomial Test Results:")
display(binomial_results)

Binomial Test Results:


,Grade,Binomial_P_Value,Status
0,A,0.824092,Accept
1,B,1.000000,Accept
2,C,0.230189,Accept
3,D,0.514699,Accept
4,E,0.751720,Accept
5,F,0.910983,Accept
6,G,0.938538,Accept


In [8]:
jeffrey_results = calculator.calculate_jeffrey_test(df_calib)
print("Jeffrey Test Results:")
display(jeffrey_results)

Jeffrey Test Results:


,Grade,Jeffrey_Lower,Jeffrey_Upper,PD_Target,Status
0,A,0.001352,0.009483,0.005,Accept
1,B,0.005857,0.016026,0.010,Accept
2,C,0.011177,0.022221,0.020,Accept
3,D,0.036260,0.057489,0.050,Accept
4,E,0.085310,0.122984,0.100,Accept
5,F,0.168587,0.238851,0.200,Accept
6,G,0.235078,0.360806,0.300,Accept


## 4. AI Reporting (Optional)

In [ ]:
# To run this, ensure you have a .env file with OPENAI_API_KEY in the root directory
run_ai = False

if run_ai:
    reporter = RiskReporter("../config.yaml")
    report = reporter.generate_report(
        auc_stats, chi_stats, binomial_results, jeffrey_results, ttest_stats
    )
    print(report)
else:
    print("Skipping AI generation (set run_ai=True to run)")

Error loading templates: 'report_instruction.j2' not found in search path: 'templates'
